# Classifying IRIS species using univariate Gaussian Classifier

**Note:** You can use built-in code for mean, variance, covariance, determinant, etc.

In [1]:
# Standard includes
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Useful module for dealing with the Gaussian density
from scipy.stats import norm, multivariate_normal #in case you use buit-in library
# installing packages for interactive graphs
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
from sklearn import datasets

### Loading the IRIS dataset

In [2]:
iris = datasets.load_iris()
X = iris.data
Y = iris.target
featurenames = ['petal_length', 'petal_width', 'sepal_length', 'sepal_width']

Confirm the dimensions:

In [3]:
X.shape, Y.shape

((150, 4), (150,))

In [4]:
# Split 150 instances into training set (trainx, trainy) of size 105 and test set (testx, testy) of size 45
np.random.seed(0)
perm = np.random.permutation(150)
trainx = X[perm[0:105],:]
trainy = Y[perm[0:105]]
testx = X[perm[105:150],:]
testy = Y[perm[105:150]]

Let's see how many training points there are from each class.

In [5]:
c0 = sum(trainy==0)
c1 = sum(trainy==1)
c2 = sum(trainy==2)

### Q1. Can you figure out how many test points there are from each class? 

In [6]:
# TODO: add your code to find how many test points there are from each class
sum(testy == 0), sum(testy == 1), sum(testy == 2)    

(17, 16, 12)

### Look at the distribution of a single feature from one of the species

Let's pick just one feature: 'petal_length'. This is the first feature, that is, number 0. Here is a *histogram* of this feature's values under species 1, along with the *Gaussian fit* to this distribution.

<img src="density.png">

In [7]:
@interact_manual( feature=IntSlider(0,0,3), label=IntSlider(0,0,2))
def density_plot(feature, label):
    plt.hist(trainx[trainy==label,feature], density=True)
    #
    mu = np.mean(trainx[trainy==label,feature]) # mean
    var = np.var(trainx[trainy==label,feature]) # variance
    std = np.sqrt(var) # standard deviation
    x_axis = np.linspace(mu - 3*std, mu + 3*std, 1000)
    plt.plot(x_axis, norm.pdf(x_axis,mu,std), 'r', lw=2)
    plt.title("Species "+str(label) )
    plt.xlabel(featurenames[feature], fontsize=14, color='red')
    plt.ylabel('Density', fontsize=14, color='red')
    plt.show()

interactive(children=(IntSlider(value=0, description='feature', max=3), IntSlider(value=0, description='label'…

### Q2. In the function **density_plot**, the code for plotting the Gaussian density focuses on the region within 3 standard deviations of the mean. Do you see where this happens? Why do you think we make this choice?

### Q3. Here's something for you to figure out: for which feature (0-3) does the distribution of (training set) values for species-2 have the *smallest* standard deviation? what is the value?

In [8]:
# modify this cell
std = np.zeros(4)
### START CODE HERE ###
Class0 = np.zeros(shape = [c0, 4])
Class1 = np.zeros(shape = [c1, 4])
Class2 = np.zeros(shape = [c2, 4])

s0 = 0
s1 = 0
s2 = 0
for i in range(len(trainy)):
    if trainy[i] == 0:
        Class0[s0] = trainx[i]
        s0 += 1
    elif trainy[i] == 1:
        Class1[s1] = trainx[i]
        s1 += 1
    elif trainy[i] == 2:
        Class2[s2] = trainx[i]
        s2 += 1



In [9]:
std = Class2.std(axis = 0)

In [21]:
print(featurenames)
print(std)

['petal_length', 'petal_width', 'sepal_length', 'sepal_width']
[0.56659062 0.28784229 0.46420337 0.28097574]


### 3. Fit a Gaussian to each class
Let's define a function that will fit a Gaussian generative model to the three classes, restricted to just a <font color = 'red'>single feature</font>.

In [11]:
def fit_generative_model(x,y,feature):
    mu = np.array([])
    mu = np.append(mu, Class0.mean(axis = 0)[feature])
    mu = np.append(mu, Class1.mean(axis = 0)[feature])
    mu = np.append(mu, Class2.mean(axis = 0)[feature])

    var = np.array([])
    var = np.append(var, Class0.var(axis = 0)[feature])
    var = np.append(var, Class1.var(axis = 0)[feature])
    var = np.append(var, Class2.var(axis = 0)[feature])

    pi = np.array([])
    pi = np.append(pi, (np.sum(trainy == 0) / len(trainy)))
    pi = np.append(pi, (np.sum(trainy == 1) / len(trainy)))
    pi = np.append(pi, (np.sum(trainy == 2) / len(trainy)))
    
    return mu, var, pi


Call this function on the feature 'petal_length'. What are the class weights?

In [12]:
feature = 0 # 'petal_length'
### START CODE HERE ###
mmm, vvv, ppp = fit_generative_model(trainx, trainy, feature)

### END CODE HERE ###

In [13]:
print(mmm)
print(vvv)
print(ppp)

[4.95757576 5.9        6.49473684]
[0.13880624 0.26176471 0.32102493]
[0.31428571 0.32380952 0.36190476]


Next, display the Gaussian distribution for each of the three classes

In [14]:
@interact_manual( feature=IntSlider(0,0,3) )
def show_densities(feature):
    mu, var, pi = fit_generative_model(trainx, trainy, feature)
    colors = ['r', 'k', 'g']
    for label in range(0,3):
        m = mu[label]
        s = np.sqrt(var[label])
        x_axis = np.linspace(m - 3*s, m+3*s, 1000)
        plt.plot(x_axis, norm.pdf(x_axis,m,s), colors[label], label="species-" + str(label))
    plt.xlabel(featurenames[feature], fontsize=14, color='red')
    plt.ylabel('Density', fontsize=14, color='red')
    plt.legend()
    plt.show()

interactive(children=(IntSlider(value=0, description='feature', max=3), Button(description='Run Interact', sty…

### Questions:

Use the widget above to look at the three class densities for each of the 4 features. Here are some questions for you:
1. For which feature (0-3) do the densities for classes 0 and 2 *overlap* the most?
2. For which feature (0-3) is class 2 the most spread out relative to the other two classes?
3. For which feature (0-3) do the three classes seem the most *separated* (this is somewhat subjective at present)?

How well can we predict the class (0, 1, 2) based just on one feature? The code below lets us find this out.

In [15]:
mu, var, pi = fit_generative_model(trainx, trainy, feature)
print(mu)
print(var)
print(pi)

[4.95757576 5.9        6.49473684]
[0.13880624 0.26176471 0.32102493]
[0.31428571 0.32380952 0.36190476]


In [16]:
def NormalPdf(x, mu, var, pi):
    contPart = 1/np.sqrt(2*np.pi*var)
    expPart = np.exp((-np.power(x - mu, 2) / (2*var)))
    px = pi * contPart * expPart
    return px

In [17]:
@interact( feature=IntSlider(0,0,3) )
def test_model(feature):
    mu, var, pi = fit_generative_model(trainx, trainy, feature)
    
    k = 3 # Labels 0,1,2,...,k
    n_test = len(testy) # Number of test points
    score = np.zeros((n_test,k)) # here it was k + 1 and I earse the 1 (k+1)
    for i in range(0,n_test):
        for label in range(0,k):
            ### START CODE HERE ###
            # Implement the formula for normal pdf. 
            # If you can't, use the built-in norm.logpdf() but to get the full grades you should implement your own  
            
            score[i,label] = NormalPdf(testx[i][feature], mu[label], var[label], pi[label])
    predictions = np.argmax(score, axis = 1) #think about using np.argmax on score[]
    
    ### END CODE HERE ###
    # Finally, tally up score
    errors = np.sum(predictions != testy)
    print ("Test error using feature " + featurenames[feature] + ": " + str(errors) + "/" + str(n_test))
    print("Test Error using feature %s: %.2f%%"%(featurenames[feature], (errors/n_test) * 100))

interactive(children=(IntSlider(value=0, description='feature', max=3), Output()), _dom_classes=('widget-inter…

In [18]:
def test_model_pred(x, y, feature):
    mu, var, pi = fit_generative_model(trainx, trainy, feature)
    
    k = 3 # Labels 0,1,2,...,k
    n_test = len(y) # Number of test points
    score = np.zeros((n_test,k)) # here it was k + 1 and I earse the 1 (k+1)
    for i in range(0,n_test):
        for label in range(0,k):
            ### START CODE HERE ###
            # Implement the formula for normal pdf. 
            # If you can't, use the built-in norm.logpdf() but to get the full grades you should implement your own  
            
            score[i,label] = NormalPdf(x[i][feature], mu[label], var[label], pi[label])
    predictions = np.argmax(score, axis = 1) #think about using np.argmax on score[]
    errors = np.sum(predictions != y)
    return errors

### Questions:
In this notebook, we are looking at classifiers that use just one out of a possible 4 features. Choosing a subset of features is called **feature selection**. In general, this is something we would need to do based solely on the *training set*--that is, without peeking at the *test set*.

For the IRIS data, compute the training error and test error associated with each choice of feature.

In [19]:
### Write your code here
error = 0
for i in range(testx.shape[1]):
    error = test_model_pred(trainx, trainy, i)
    print(error)
    print("training error for feature #%d = %.2f%%"%(i , 100*error/len(trainy)))


    
print()    
for i in range(testx.shape[1]):
    error = test_model_pred(testx, testy, i)
    print(error)
    print("test error for feature #%d = %.2f%%"%(i , 100*error/len(testy)))
    



26
training error for feature #0 = 24.76%
45
training error for feature #1 = 42.86%
4
training error for feature #2 = 3.81%
4
training error for feature #3 = 3.81%

12
test error for feature #0 = 26.67%
20
test error for feature #1 = 44.44%
3
test error for feature #2 = 6.67%
2
test error for feature #3 = 4.44%


Based on your findings, answer the following questions:
* Which two features have the lowest training error? List them in order (best first).
* Which two features have the lowest test error? List them in order (best first).